In [1]:
!pip install sentence_transformers
!pip install konlpy

In [2]:
import numpy as np
import itertools

from konlpy.tag import Hannanum
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [3]:
hannanum = Hannanum()
#한국어 불용어
stop_words="아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓"
stop_words=stop_words.split(' ')

# SBERT를 로드
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

In [13]:
def max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [14]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [15]:
def keyword_extraction(doc_with_newlines):
    doc=doc_with_newlines.replace("\n", "")
    doc=doc.replace("'", "")
    doc=doc.replace('"', '')
    #문장을 분리하였지만, 아직 앞뒤 공백이 있음
    temp_texts =  doc.split(". ")

    #앞뒤 공백이 제거된 문자열들이 저장되는 list
    texts=[]
    for text in temp_texts:
        text = text.strip()
        if text is not '':
            texts.append(text)
    
    # 품사 태깅
    tokenized_doc = hannanum.pos(doc)

    #print(tokenized_doc)
    # 형태소 분석기를 통해 명사 추출한 문서를 만듦
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc
                        if (word[1] == 'N' or word[1] == 'F')
                        and not word[0].isdigit() 
                        and len(word[0])>1
                        and word[0] not in stop_words])

    n_gram_range = (1, 1)
    # CountVectorizer를 사용하여 n-gram을 추출(n_gram_range에 해당하는)
    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names()

    #doc를 임베딩
    doc_embedding = model.encode([doc])
    #n_gram들을 임베딩
    candidate_embeddings = model.encode(candidates)
    
    # nr_candidate가 클수록, 키워드간의 유사성이 최소화 됨, 문서와 키워드 사이의 유사성은 극대화
    mss_keywords=max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=15, nr_candidates=20)
    
    # diversity값이 클수록, 키워드간의 유사성이 최소화 됨, 문서와 키워드 사이의 유사성은 극대화
    mmr_keywords=mmr(doc_embedding, candidate_embeddings, candidates, top_n=15, diversity=0.2)

    #공백이 있는 후보 키워드들을 저장
    space_candidate_keywords=set([])
    space_candidate_keywords = space_candidate_keywords.union(mss_keywords)
    space_candidate_keywords = space_candidate_keywords.union(mmr_keywords)

    #후보 키워드들의 공백을 없앰
    candidate_keywords=set([])
    for keyword in space_candidate_keywords:
        keyword = keyword.replace(' ', '')
        candidate_keywords.add(keyword)
    
    #후보키워드들 중  상위 30개 구함
    doc_without_space=doc.replace(' ', '')
    count_dict = set([(token, doc_without_space.count(token)) for token in candidate_keywords ])
    ranked_words = sorted(count_dict, key=lambda x:x[1], reverse=True)[:30]

    predict_keywords = list([keyword for keyword, freq in ranked_words])

    return predict_keywords

In [16]:
#test_data를 파일로부터 읽어옴

news_file = open("news_data.txt", 'r')
target_file = open("target_data.txt", 'r')

news_data = []
target_data = []


while True:
    news_line = news_file.readline()
    target_line = target_file.readline()
    if not news_line or not target_line: 
        break
    news_data.append(news_line.replace('\n', ""))
    target_data.append(target_line.replace('\n', ""))

news_file.close()
target_file.close()

average=0.0
for i in range(0, len(news_data)):
    #모델에 넣고 돌려서 나온 키워드 리스트 
    predict_keywords = keyword_extraction(news_data[i])
    #실제 답인 키워드 리스트
    target_keywords=target_data[i].split(' ')
    #실제 답인 키워드들을 돌면서
    score = 0.0
    for target_keyword in target_keywords:
        #해당 키워드가 예측한 키워드 리스트에 있는지 체크하며 있으면 개수 늘림
        if target_keyword in predict_keywords:
            score+=1.0
    average += score/len(target_keywords)
    print("index", i, "->", score/len(target_keywords))

average /= len(news_data)

print("결과: ", average)

index 0 -> 0.0
index 1 -> 0.6
index 2 -> 0.0
index 3 -> 0.15384615384615385
index 4 -> 0.1111111111111111
index 5 -> 0.0
index 6 -> 0.14285714285714285
index 7 -> 0.375
index 8 -> 0.14285714285714285
index 9 -> 0.25
index 10 -> 0.0
index 11 -> 0.14285714285714285
index 12 -> 0.3
index 13 -> 0.1
index 14 -> 0.3333333333333333
index 15 -> 0.16666666666666666
index 16 -> 0.0
index 17 -> 0.1111111111111111
index 18 -> 0.0
index 19 -> 0.0
index 20 -> 0.5
index 21 -> 0.16666666666666666
index 22 -> 0.4
index 23 -> 0.2
index 24 -> 0.375
index 25 -> 0.0
index 26 -> 0.5714285714285714
index 27 -> 0.25
index 28 -> 0.14285714285714285
index 29 -> 0.125
index 30 -> 0.2
index 31 -> 0.09090909090909091
index 32 -> 0.08333333333333333
index 33 -> 0.0
index 34 -> 0.1111111111111111
index 35 -> 0.3333333333333333
index 36 -> 0.4
index 37 -> 0.2857142857142857
index 38 -> 0.08333333333333333
index 39 -> 0.5384615384615384
index 40 -> 0.4
index 41 -> 0.0
index 42 -> 0.14285714285714285
index 43 -> 0.25
i

index 348 -> 0.3333333333333333
index 349 -> 0.3
index 350 -> 0.08333333333333333
index 351 -> 0.18181818181818182
index 352 -> 0.3
index 353 -> 0.0
index 354 -> 0.0
index 355 -> 0.09090909090909091
index 356 -> 0.0
index 357 -> 0.1
index 358 -> 0.2727272727272727
index 359 -> 0.0
index 360 -> 0.42857142857142855
index 361 -> 0.2
index 362 -> 0.3333333333333333
index 363 -> 0.16666666666666666
index 364 -> 0.2
index 365 -> 0.25
index 366 -> 0.0
index 367 -> 0.07142857142857142
index 368 -> 0.07692307692307693
index 369 -> 0.08333333333333333
index 370 -> 0.36363636363636365
index 371 -> 0.25
index 372 -> 0.14285714285714285
index 373 -> 0.2
index 374 -> 0.0
index 375 -> 0.16666666666666666
index 376 -> 0.0
index 377 -> 0.3333333333333333
index 378 -> 0.0
index 379 -> 0.1111111111111111
index 380 -> 0.23076923076923078
index 381 -> 0.0
index 382 -> 0.16666666666666666
index 383 -> 0.0
index 384 -> 0.2
index 385 -> 0.08333333333333333
index 386 -> 0.2
index 387 -> 0.25
index 388 -> 0.142

index 691 -> 0.16666666666666666
index 692 -> 0.0
index 693 -> 0.0
index 694 -> 0.1111111111111111
index 695 -> 0.0
index 696 -> 0.18181818181818182
index 697 -> 0.14285714285714285
index 698 -> 0.18181818181818182
index 699 -> 0.15384615384615385
index 700 -> 0.08333333333333333
index 701 -> 0.0
index 702 -> 0.0
index 703 -> 0.125
index 704 -> 0.16666666666666666
index 705 -> 0.0
index 706 -> 0.2
index 707 -> 0.08333333333333333
index 708 -> 0.0
index 709 -> 0.125
index 710 -> 0.0
index 711 -> 0.3333333333333333
index 712 -> 0.14285714285714285
index 713 -> 0.15384615384615385
index 714 -> 0.07692307692307693
index 715 -> 0.4
index 716 -> 0.0
index 717 -> 0.3333333333333333
index 718 -> 0.0
index 719 -> 0.18181818181818182
index 720 -> 0.0625
index 721 -> 0.0
index 722 -> 0.07692307692307693
index 723 -> 0.1
index 724 -> 0.18181818181818182
index 725 -> 0.2
index 726 -> 0.14285714285714285
index 727 -> 0.14285714285714285
index 728 -> 0.0
index 729 -> 0.14285714285714285
index 730 -> 

index 1029 -> 0.1111111111111111
index 1030 -> 0.1111111111111111
index 1031 -> 0.0
index 1032 -> 0.0
index 1033 -> 0.2727272727272727
index 1034 -> 0.0
index 1035 -> 0.0
index 1036 -> 0.0
index 1037 -> 0.25
index 1038 -> 0.0
index 1039 -> 0.18181818181818182
index 1040 -> 0.18181818181818182
index 1041 -> 0.0
index 1042 -> 0.36363636363636365
index 1043 -> 0.0
index 1044 -> 0.4
index 1045 -> 0.09090909090909091
index 1046 -> 0.1111111111111111
index 1047 -> 0.09090909090909091
index 1048 -> 0.0
index 1049 -> 0.0
index 1050 -> 0.0
index 1051 -> 0.1111111111111111
index 1052 -> 0.18181818181818182
index 1053 -> 0.08333333333333333
index 1054 -> 0.4
index 1055 -> 0.1
index 1056 -> 0.125
index 1057 -> 0.0
index 1058 -> 0.23076923076923078
index 1059 -> 0.0
index 1060 -> 0.4
index 1061 -> 0.14285714285714285
index 1062 -> 0.0
index 1063 -> 0.0
index 1064 -> 0.0
index 1065 -> 0.0
index 1066 -> 0.2857142857142857
index 1067 -> 0.0
index 1068 -> 0.2
index 1069 -> 0.07692307692307693
index 107

index 1377 -> 0.0
index 1378 -> 0.0
index 1379 -> 0.125
index 1380 -> 0.2857142857142857
index 1381 -> 0.2727272727272727
index 1382 -> 0.0
index 1383 -> 0.0
index 1384 -> 0.09090909090909091
index 1385 -> 0.18181818181818182
index 1386 -> 0.0
index 1387 -> 0.0
index 1388 -> 0.14285714285714285
index 1389 -> 0.25
index 1390 -> 0.1
index 1391 -> 0.42857142857142855
index 1392 -> 0.0
index 1393 -> 0.6
index 1394 -> 0.25
index 1395 -> 0.125
index 1396 -> 0.0
index 1397 -> 0.0
index 1398 -> 0.0
index 1399 -> 0.09090909090909091
index 1400 -> 0.3333333333333333
index 1401 -> 0.4
index 1402 -> 0.14285714285714285
index 1403 -> 0.6
index 1404 -> 0.2
index 1405 -> 0.0
index 1406 -> 0.5
index 1407 -> 0.0
index 1408 -> 0.14285714285714285
index 1409 -> 0.0
index 1410 -> 0.0
index 1411 -> 0.0
index 1412 -> 0.0
index 1413 -> 0.0
index 1414 -> 0.3333333333333333
index 1415 -> 0.0
index 1416 -> 0.14285714285714285
index 1417 -> 0.0
index 1418 -> 0.0
index 1419 -> 0.14285714285714285
index 1420 -> 0.

index 1718 -> 0.3333333333333333
index 1719 -> 0.125
index 1720 -> 0.1
index 1721 -> 0.0
index 1722 -> 0.0
index 1723 -> 0.125
index 1724 -> 0.125
index 1725 -> 0.0
index 1726 -> 0.2857142857142857
index 1727 -> 0.0
index 1728 -> 0.125
index 1729 -> 0.2
index 1730 -> 0.5714285714285714
index 1731 -> 0.14285714285714285
index 1732 -> 0.1111111111111111
index 1733 -> 0.08333333333333333
index 1734 -> 0.4
index 1735 -> 0.1
index 1736 -> 0.16666666666666666
index 1737 -> 0.125
index 1738 -> 0.3
index 1739 -> 0.0
index 1740 -> 0.0
index 1741 -> 0.2
index 1742 -> 0.18181818181818182
index 1743 -> 0.3333333333333333
index 1744 -> 0.2857142857142857
index 1745 -> 0.2222222222222222
index 1746 -> 0.0
index 1747 -> 0.0625
index 1748 -> 0.3
index 1749 -> 0.14285714285714285
index 1750 -> 0.25
index 1751 -> 0.0
index 1752 -> 0.14285714285714285
index 1753 -> 0.2
index 1754 -> 0.07142857142857142
index 1755 -> 0.0
index 1756 -> 0.42857142857142855
index 1757 -> 0.3333333333333333
index 1758 -> 0.16